# Robust Lot Sizing Problem

The Lot Sizing problem is a fundamental optimization challenge in operations management, focusing on finding the best quantities to produce or order in each period to meet demand while minimizing costs. Over a fixed planning horizon, the problem involves known product demand and associated costs like setup, production, inventory holding, and backorders. Key decisions include determining production quantities and initiation timings. The primary goal is to devise a production plan that efficiently meets demand each period. In its robust form, the problem considers uncertain production yields, fluctuating within a range defined by its nominal value and maximum deviation.

In this example, we aim to solve the Robust LSP provided in Robust optimization for lot-sizing problems under yield uncertainty (paula metzker, simon thevenin). Let us consider a single - item multi period uncapacitated LSP with backorder and production yield determines the quantity to produce in each period of the finite planning horizon $T$. We are given the setup cost $\vec{s}$, the inventory holding cost $\vec{h}$, the backorder cost $\vec{b}$ and the demand $\vec{d}$ for each time period in $T$. The model contains the decision variables $X_t$, the lot size to be produced, and the setup decision $Y_t$. The strictly positive uncertain production yield is defined as $\bar{p}$. 

The robust problem can be optimization as follows:

$$
\begin{aligned}
& \text{minimize} \quad s^TY + v^TX + H \\
& \text{such that} \\
& H_t \geq h_t[ \bar{p}^TX_t - d], \\
& H_t \geq -b_t[ \bar{p}^TX_t - d], \\
& X_t \leq M_tY_t, \\
& X_t, H_t \geq 0, \\
\end{aligned}

$$

To solve this problem, we first import the required packages.

In [11]:
import numpy as np
import lropt
import cvxpy as cp

We start by defining the relevant constants and creating variables. 

In [12]:
np.random.seed(1)
T = 5 # Using 5 periods as an example
s = np.random.rand(T)  # Setup costs
v = np.random.rand(T)  # Unit production costs
h = np.random.rand(T)  # Inventory holding costs
b = np.random.rand(T)  # Backorder costs
d = np.random.randint(10, 20, size=T)  # Demand

# Large positive number for setup constraint
M = np.full(T, 1000)


X = cp.Variable(T, nonneg=True)  # Production quantities
Y = cp.Variable(T, boolean=True)  # Setup decision variables (binary)
H = cp.Variable(T, nonneg=True)  # Cost terms for inventory and backorder costs
NUM_DEC = 3 


In the following code cell, we have defined the uncertain parameter p as a part of the Budget Uncertainty set.


In [13]:
p = lropt.UncertainParameter(T, uncertainty_set = lropt.Budget(), nonneg = True)

Here, we have defined our objective and constraints. Finally, we can solve the problem and get the optimal value. 

In [14]:
objective = cp.Minimize(cp.sum(s @ Y + v @ X + H))
constraints = [
H >= h * cp.sum(p @ X - d),
H >= (-b) * cp.sum(p @ X - d),
X <= M@Y
]

In [15]:
prob = lropt.RobustProblem(objective, constraints)
prob.solve()
print(f"The robust optimal value using Budget uncertainty is {round(prob.solve(), NUM_DEC)}")

The robust optimal value using Budget uncertainty is 36.176
